Initial code to investigate basic RL model for price minimisation of HVAC system.

States:
    - Room temp
    - External temp
    - Elec price

Action:
    - Choose temp setpoint

Training methodology

1. For each hourly timestep:
    - Load internal temp, external temp, and current electricity price (states)
    - Choose temperature setpoint (action)
    - Simulate 60 mins of PI controller working
        - Re calculating required heat load (Q_hvac) at each minute
        - Step building thermal model 1 min (simulation_enviornment) to calc new internal temperature
    - Final internal temperature calculation from thermal model (60th value) is internal temperature for next step
    - Calculate reward

Measured data required:
    - Only need measurements for external temperature and electricity price
    - Internal temperature can be initialised to any value close to desired setpoint,
      then calculated from building thermal model
    - Temperature setpoint chosen by RL agent
    - HVAC heat load calculated from PI controller


In [1]:
import numpy as np
import pandas as pd
import rl_hvac_training as rl
import simulation_environment as sim_env
from pi_controller import PIController, apply_system_constraints

In [2]:
#Prices May-July 2022
realprices = pd.read_csv("./input_data/3 year N2EX elec Prices.csv")
testprices = realprices.iloc[20424:22632]

#Temperatures May-July 2023
external_temps = pd.read_csv("./input_data/measured_external_temps_01052023-01082023.csv", index_col = 0)

In [3]:
agent = rl.EpsilonGreedy(0.01)
agent.run(0.4,0.6,external_temps,testprices)

(51, 30, 65, 30)


IndexError: index 51 is out of bounds for axis 0 with size 51

In [4]:
elec_prices = pd.read_excel("./input_data/fake_elec_prices.xlsx")

#  PI controller example
T_sp = 25 # setpoint temperature
T_internal = 18 # internal temperature
q_max = 500000 # [W] maximum combined heat load for AHUs

pi_controller = PIController(max_heat_duty=q_max)
pi_signal = pi_controller.pi_signal(T_sp, T_internal)
q_hvac = apply_system_constraints(pi_signal, q_max) # [W] HVAC heat load
# thermal model example
rc_vals = [0.0496, 0.5862, 0.1383, 59.01, 247.8, 0.1576, 0.0289, 1749100, 388300]
sales_area_dims = [117, 64.866, 9] # width, length, height
# initialise all RC model temperatures at the same value. Read MPC paper RC schematic to understand notation.
T_init = [16]*13 # ["T_w12", "T_w14", "T_w22", "T_w24", "T_w32", "T_w34", "T_w42", "T_w44", "T_r2", "T_r4", "T_i1", "T_i2", "T_in"]
thermal_model = sim_env.ThermalNetwork(rc_vals,sales_area_dims,T_init,dt=60) # number of second for each timestemp. Needs to be 60

"""
model_inputs is a vector (np.array), each column is detailed below with an example:
    - [T_external_wall_1, T_external_wall_2, T_external_wall_3, T_external_wall_4, T_external_roof, q_hvac]
    - T_external_wall_x and T_external_roof are all the same value of the measured external temperature
    - need to resample and interpolate external temperatures to get minutely values. 
"""
model_inputs = np.array([external_temps.iloc[0,0]] * 5 + [q_hvac]) # one example timestep
next_timestep_temps = thermal_model.calcNextTimestep(model_inputs) # ["T_w12", "T_w14", "T_w22", "T_w24", "T_w32", "T_w34", "T_w42", "T_w44", "T_r2", "T_r4", "T_i1", "T_i2", "T_in"]


In [9]:
next_timestep_temps[-1]

16.280718140126208

In [4]:
rl.discretise(testprices, 1, 100)

65

In [6]:
np.max(testprices.iloc[:,1])

0.64764

In [ ]:
np.max(external_temps.iloc[:,0]) - np.min(external_temps.iloc[:,0])